In [22]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import skimage as sk


In [23]:
def rgb2gray(image):
    return cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)


def gauss(image):
    return cv2.GaussianBlur(image, (5, 5), 0)


def canny_edges(image):
    edges = cv2.Canny(image, 50, 150)
    return edges


def hor_lines(edges_image):

    kernel = np.array([[-1, -1, -1], [2, 2, 2], [-1, -1, -1]])

    output = cv2.filter2D(edges_image, -1, kernel)
    return output


def sin_scale(arr):

    N = len(arr)
    x = np.linspace(0, np.pi, N)
    scaler = np.sin(x)
    scaled = np.multiply(arr, scaler)

    return scaled


def rowsums(edges_im):
    rowsums = list()
    for row in edges_im:
        rowsums.append(np.sum(row))
    return rowsums


def scaled_rowsums(edges_im):
    return sin_scale(rowsums(edges_im))


def find_ROI_border(frame):

    gray = rgb2gray(frame)
    canny = canny_edges(gray)
    hor_edges = hor_lines(canny)
    top_border = np.argmax(scaled_rowsums(hor_edges))

    return top_border


def display_with_border(frame, border):

    with_line = cv2.line(np.copy(frame), (0, border),
                         (703, border), (255, 0, 0), 1)
    cv2.imshow('hey', cop)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


def cut_border(frame, border):
    copy = np.copy(frame)
    copy = copy[:][border+1:][:]
    return copy


In [24]:
def add_gaussian(image):
    scaled_im = frame/255.0
    return sk.util.random_noise(scaled_im, mode='gaussian', clip=True)


def add_sp(image):
    scaled_im = frame/255.0
    sk.util.random_noise(scaled_im, mode='s&p')


In [25]:
cap = cv2.VideoCapture('april21.avi')
cap.set(cv2.CAP_PROP_POS_FRAMES, 150)
ret, frame = cap.read()


In [26]:
ind = find_ROI_border(frame)
cop = cut_border(frame, ind)


In [70]:
hsv = cv2.cvtColor(cop, cv2.COLOR_RGB2HSV)
# define range of white color in HSV
# change it according to your need !
lower_white = np.array([0, 0, 180], dtype=np.uint8)
upper_white = np.array([172, 111, 255], dtype=np.uint8)

# Threshold the HSV image to get only white colors
mask = cv2.inRange(hsv, lower_white, upper_white)
# Bitwise-AND mask and original image
res = cv2.bitwise_and(cop, cop, mask=mask)

cv2.imshow('frame', frame)
cv2.imshow('mask', mask)
cv2.imshow('res', res)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [28]:

# YCrCb = cv2.cvtColor(cop, cv2.COLOR_RGB2YCR_CB)
# HLSc = cv2.cvtColor(cop, cv2.COLOR_RGB2HLS)
# cd = YCrCb[:, :, 1]
# m = np.mean(cd)
# indexes = cd < 0.9*m


# cd1 = HSLc[:,:,2]
# m1 = np.mean(cd1)
# indexes1 = cd1 > 1.5*m1

# seg1 = rgb2gray(cop)
# seg1[indexes1] = 255
# seg1[np.bitwise_not(indexes1)] = 0

# cv2.imshow('hey', seg1)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [29]:
# seg = rgb2gray(cop)
# seg[indexes] = 255
# seg[np.bitwise_not(indexes)] = 0

# cv2.imshow('hey', seg)
# cv2.waitKey(0)
# cv2.destroyAllWindows()


In [71]:
hough_lines = cv2.HoughLinesP(
    mask, rho=1, theta=1*np.pi/180, threshold=50, minLineLength=100, maxLineGap=50)
line_im = np.copy(cop)
for line in hough_lines:
    line = line[0][:]
    pt1 = (line[0], line[1])
    pt2 = (line[2], line[3])
    cv2.line(line_im, pt1, pt2, (0, 0, 255), 2)


cv2.imshow('hey', line_im)
cv2.waitKey(0)
cv2.destroyAllWindows()
